This section will check the below
  - Abnormal daily revenue patterns
  - Silent data issues

In [0]:
# ------------------------------------------------------------
# STEP 1: Import required ML library
# IsolationForest is used for unsupervised anomaly detection
# ------------------------------------------------------------
from sklearn.ensemble import IsolationForest


# ------------------------------------------------------------
# STEP 2: Read Gold-level business KPI table from Databricks
# This table contains daily aggregated revenue metrics
# ------------------------------------------------------------
gold_df = spark.table(
    "sales_lakehouse.gold.daily_sales_kpi"
)


# ------------------------------------------------------------
# STEP 3: Convert Spark DataFrame to Pandas
# ML libraries like scikit-learn require Pandas input
# Data volume is small here (daily aggregates), so safe to convert
# ------------------------------------------------------------
pdf = gold_df.toPandas()


# ------------------------------------------------------------
# STEP 4: Initialize the AI anomaly detection model
# contamination = expected percentage of anomalies in data
# random_state = ensures reproducible results
# ------------------------------------------------------------
model = IsolationForest(
    contamination=0.025,   # ~3% days expected to be abnormal
    random_state=42
)


# ------------------------------------------------------------
# STEP 5: Train the model and detect anomalies
# Model learns normal revenue patterns and flags deviations
# Output:
#   1  = normal data point
#  -1  = anomaly
# ------------------------------------------------------------
pdf["revenue_anomaly_flag"] = model.fit_predict(
    pdf[["daily_revenue"]]
)


# ------------------------------------------------------------
# STEP 6: Persist AI quality results back to Databricks
# This creates a reusable quality monitoring table
# ------------------------------------------------------------
quality_df = spark.createDataFrame(pdf)

(
    quality_df.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("sales_lakehouse.quality.daily_revenue_anomalies")
)


# ------------------------------------------------------------
# STEP 7: View only anomalous records for investigation
# These rows i


Revenue Trend with Anomalies (Visualization)

In [0]:
import matplotlib.pyplot as plt

# Read quality table
pdf = spark.table(
    "sales_lakehouse.quality.daily_revenue_anomalies"
).toPandas()

# Split normal and anomaly records
normal_df = pdf[pdf["revenue_anomaly_flag"] == 1]
anomaly_df = pdf[pdf["revenue_anomaly_flag"] == -1]

# Create the plot
plt.figure(figsize=(14, 6))

# Plot normal revenue trend
plt.plot(
    normal_df["order_date"],
    normal_df["daily_revenue"],
    label="Normal Revenue",
    linewidth=2
)

# Highlight anomaly points
plt.scatter(
    anomaly_df["order_date"],
    anomaly_df["daily_revenue"],
    label="Anomaly",
    s=60
)

# Chart labels and title
plt.title("Daily Revenue with AI-Detected Anomalies")
plt.xlabel("Order Date")
plt.ylabel("Daily Revenue")
plt.legend()
plt.grid(True)

# Display chart
plt.show()


Anomaly Summary (KPI View)
  
  How many days are affected?
  How serious is it?

In [0]:

from pyspark.sql.functions import count, when, col

summary_df = (
    spark.table("sales_lakehouse.quality.daily_revenue_anomalies")
    .withColumn(
        "status",
        when(col("revenue_anomaly_flag") == -1, "Anomaly")
        .otherwise("Normal")
    )
    .groupBy("status")
    .agg(count("*").alias("number_of_days"))
)

summary_df.display()
